# Let's Build a Quant Trading Strategy - Part 3

In [ ]:
# y_hat = model(x)
# orders = strategy(y_hat)
# execute(orders)

In [ ]:
# 1st video = Research
# 2nd video = Strategy
# 3rd video = Implementation

In [ ]:
# By the end of this video, we will have wrote the code for our strategy and model that we developed in part 2 and 1 respectively - put it live with real money and see what happens.
# I will do a follow up video to see how it performs. 
# Even if it doesn't perform as expected, we can learn from it. 
# We might experience model drift - where where the pattern it learned starts to no longer hold true. You might hear this be called alpha decay - which is your profits decay because of model drift. 




In [ ]:
# This will video will be more software engineering now as we have to code it all together.

# 1. Create the strategy using the decisions we made in part 2
# 2. Create Strategy API that allows to create new strategies easily. Avoid copy and pasting code between strategies.
# 3. Create execution API to connect to an exchange - which I won't make public because I don't want people to tweak parameters willy nilly and run it then lose everything.


## The fundamental building block: Tick

In [ ]:
from abc import ABC, abstractmethod
from typing import Generic, Optional, TypeVar

T = TypeVar('T')  # input type
R = TypeVar('R')  # output type

class Tick(ABC, Generic[T, R]):
    @abstractmethod
    def on_tick(self, val: T) -> Optional[R]:
        """Handle a new tick and optionally return a result."""
        pass

## The fundamental data structure: Window

In [11]:
from collections import deque
from typing import Deque, Optional

class Window(Tick[T, T], Generic[T]):
    def __init__(self, n: int):
        self._data: Deque[T] = deque(maxlen=n)

    def on_tick(self, val: T) -> Optional[T]:
        """Append a value and return the oldest value dropped (if any)."""
        dropped = None
        if len(self._data) == self.capacity():
            dropped = self._data[0]
        self._data.append(val)
        return dropped

    def capacity(self) -> int:
        return self._data.maxlen
    
    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(capacity={self._data.maxlen}, values={list(self._data)})"

In [18]:
w = Window(3)
w.on_tick(1)
w

Window(capacity=3, values=[1])

In [21]:
w = Window(3)
w.on_tick(1)
w.on_tick(2)
w

Window(capacity=3, values=[1, 2])

In [24]:
w = Window(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
w


Window(capacity=3, values=[1, 2, 3])

In [28]:
w = Window(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
old_v = w.on_tick(4)

(old_v, w)

(1, Window(capacity=3, values=[2, 3, 4]))

In [29]:
w = Window(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
w.on_tick(4)
old_v = w.on_tick(5)

(old_v, w)

(2, Window(capacity=3, values=[3, 4, 5]))

In [ ]:
class DequeWindow(Tick[T, T], Generic[T]):
    def __init__(self, n: int):
        self._data: Deque[T] = deque(maxlen=n)

    def on_tick(self, val: T) -> Optional[T]:
        """Append a value and return the oldest value dropped (if any)."""
        dropped = None
        if len(self._data) == self.capacity():
            dropped = self._data[0]
        self._data.append(val)
        return dropped

    def capacity(self) -> int:
        return self._data.maxlen
    
    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(capacity={self._data.maxlen}, values={list(self._data)})"

In [ ]:
import torch 

class TensorWindow(Tick[T, T]):
    def __init__(self, n: int):
        self._capacity = n
        self._data = torch.zeros(n)  # 1D fixed-length tensor
        self._count = 0              # how many elements we’ve seen so far

    def on_tick(self, val: T) -> Optional[T]:
        """Append a scalar or 1D tensor and return the dropped value (if any)."""
        if val.numel() != 1:
            raise ValueError("Only scalar tensors are supported for now.")
        
        dropped = None
        if self._count >= self._capacity:
            dropped = self._data[0]  # copy the oldest value
            # shift left
            self._data[:-1] = self._data[1:]
            self._data[-1] = val
        else:
            self._data[self._count] = val
            self._count += 1
        
        return dropped

    def capacity(self) -> int:
        return self._capacity

    def values(self) -> torch.Tensor:
        """Return current filled values as a tensor view."""
        return self._data[:self._count]

    def __repr__(self) -> str:
        cls = self.__class__.__name__
        vals = self.values().tolist()
        return f"{cls}(capacity={self._capacity}, values={vals})"

In [ ]:
class Last(Tick[T, T], Generic[T]):
    def __init__(self):
        self._value: Optional[T] = None

    def on_tick(self, val: T) -> Optional[T]:
        """Append a value and return the oldest value dropped (if any)."""
        self._value = val
        return val

    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(value={self._value})"
    


In [31]:
last_val = Last()
last_val

Last(value=None)

In [32]:
last_val = Last()
last_val.on_tick(1)
last_val.on_tick(2)
last_val.on_tick(3)
last_val

Last(value=3)

In [33]:
class First(Tick[T, T], Generic[T]):
    def __init__(self):
        self._value: Optional[T] = None

    def on_tick(self, val: T) -> Optional[T]:
        """Append a value and return the oldest value dropped (if any)."""
        if self._value is None:
            self._value = val
        return self._value

    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(value={self._value})"

In [34]:
first = First()
first

First(value=None)

In [36]:
first = First()
first.on_tick(1)
first.on_tick(2)
first.on_tick(3)
first

First(value=1)